# Lab 3

Authors: Riley Galante, Jeffrey Taylor, Eric Bernard, Austin Hayden

# Prediction Task and Business Application

### Business Understanding

This data can be useful in determining the age and content rating of a video game with the goal of being able to classify a video game with the appropriate rating it would receive before entering the market.

This dataset contains 1,895 unique video games that have already been rated by the Entertainment Software Rating Board (ESRB). The ESRB rates video games on the following scale: Rating Pending (RP), Early Childhood (EC), Everyone (E), Everyone 10+ (E 10+), Teen (T), Mature (M), and Adult (A). In this dataset, only games rated E, E 10+, T, and M are represented, so our classifier will be limited to these ratings. It also contains many important features that contribute to the certain rating a video game receives. These features are represented as binary values where 0 relates to the video game not having that characteristic, and 1 relates to the video game having that attribute. Some of the important features that will help classify the video game include Alcohol References, Blood and Gore, Crude Humor, Language, Nudity, Use of Drugs and Alcohol, Violence, etc.

The ratings given to the video game determine age restrictions on who can purchase the game and also the intended audience of the video game.

### Third-party Interest

Because our end goal is to be able to classify a video game to an ESRB rating based on whether the game has certain features or not, game developers would be interested in this classification algorithm. The algorithm would be useful to them because they would be able to have insight into the rating the game will receive before it is given its official rating by the ESRB. Therefore, game developers can tweak their game according to the intended audience of the game. For example, if a game is being developed for everyone but receives a rating of Teen from our algorithm, then the game developers could look at removing or reducing certain features of the game to bring down the rating. Furthermore, this algorithm will be useful in marketing the video game. By having an insight into the rating the game will receive by the ESRB, game developers can go ahead and begin marketing the video game to the audience of the rating it is classified as, while it still has not been officially rated.

The use of this algorithm would be offline since there is no real time advantage of having the model deployed. Game developers will use the classification algorithm when they have a game ready to be rated or whenever they want to test what a certain combination of features of a game will be rated as.

### Measure of Success

Since the classification algorithm is not for the official rating and only used as an insight into the official rating, we think that a success rate of 80% or greater would be considered useful for game developers. Having a success rate of greater than 80% will give game developers confidence in our algorithm that the rating it is classified as will most likely be the official rating it receives.

Any misclassifications will not result in major concern to the game developers. The only real negative to a misclassification would be a waste in resources marketing to the wrong audience. Misclassifications are expected since we are trying to predict the decision of the ESRB. As long as misclassifications are minimal, game developers will still have faith in our classification algorithm.

Dataset source: https://www.kaggle.com/imohtn/video-games-rating-by-esrb

# Data Preparation

In [311]:
import pandas as pd
import numpy as np

df = pd.read_csv('Video_games_esrb_rating.csv')

# map the esrb string values to integers for our target
rating = []

for i in df['esrb_rating']:
    if(i == 'E'):
        rating.append(0)
    elif(i == 'ET'):
        rating.append(1)
    elif(i == 'T'):
        rating.append(2)
    else:
        rating.append(3)
        
del df['title'] #get rid of unnecessary data
del df['console']
del df['esrb_rating'] #get rid of original column since its been mapped now

print(df.dtypes)
print("\n{:+^60s}\n".format(""))
print(df.info())

alcohol_reference           int64
animated_blood              int64
blood                       int64
blood_and_gore              int64
cartoon_violence            int64
crude_humor                 int64
drug_reference              int64
fantasy_violence            int64
intense_violence            int64
language                    int64
lyrics                      int64
mature_humor                int64
mild_blood                  int64
mild_cartoon_violence       int64
mild_fantasy_violence       int64
mild_language               int64
mild_lyrics                 int64
mild_suggestive_themes      int64
mild_violence               int64
no_descriptors              int64
nudity                      int64
partial_nudity              int64
sexual_content              int64
sexual_themes               int64
simulated_gambling          int64
strong_janguage             int64
strong_sexual_content       int64
suggestive_themes           int64
use_of_alcohol              int64
use_of_drugs_a

### Final Dataset Description
For this dataset, the first 31 columns are descriptors that go into the esrb rating for a video game. All those colums are binary data, where 1 represents a video game that has that particular attribute and 0 means it does not. Because the esrb rating was in string format, we one-hot encoded it so that the logistic regression would be able to successfully predict this categorical data. This is seen in the rating_E, rating_ET, rating_M, and rating_T columns that were added, representing E for everyone, E 10+, M for Mature, and T for Teen respectively. Additionally, we removed unnecessary information from the dataset that would not help our prediction, like the titles and console of the video games.

In [312]:
from sklearn.model_selection import ShuffleSplit

# E, ET, M, T = df['rating_E'].to_numpy(), df['rating_ET'].to_numpy(), df['rating_M'].to_numpy(), df['rating_T'].to_numpy()

#stack rating encodings into one array for target array
# y = np.vstack((E, ET, M, T)).T
# yf = list(map(lambda i: str(i), y))
# t_str = ''
# for x in range(len(y)):
#     t_str = yf[x]
#     t = t_str.replace('[','')
#     t = t_str.replace(']','')
#     t = t_str.replace(' ','')
#     print(int(t[0],2))

y = np.array(rating).T

#remaining columns in dataset are used for X
X = df.to_numpy()

print(y.shape)
print(y)

#set up shuffle split using example
num_cv_iterations = 3
num_instances = len(y)
cv_object = ShuffleSplit(n_splits = num_cv_iterations, test_size = 0.2)
                         
print(cv_object)
print(num_instances)

(1895,)
[0 1 3 ... 0 2 0]
ShuffleSplit(n_splits=3, random_state=None, test_size=0.2, train_size=None)
1895


### Training and Testing Split

The 80/20 train-test split is appropriate for our data set because it is not dependent on ordering or a certain time factor. Regardless of which are chosen for training or testing, all instances in our dataset use just the particular descriptors for that game to determine the rating, nothing more.

# Implementation

### Basic Binary Logisitic Regression that Other Classes will Inherit
Binary Log Reg, or versions which will inherit off it, will be used once problem is simplified to binary as part of one vs all structure in the main multi class classifier.

In [313]:
# basic binary logistic regression with gradient descent from class examples
from scipy.special import expit
class BinaryLogisticRegression:
    def __init__(self, eta, norm, iterations=20, C=0.001):
        self.eta = eta
        self.norm = norm
        self.iters = iterations
        self.C = C
        # internally we will store the weights as self.w_ to keep with sklearn conventions
        
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'Binary Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained Binary Logistic Regression Object'
        
    # convenience, private:
    @staticmethod
    def _add_bias(X):
        return np.hstack((np.ones((X.shape[0],1)),X)) # add bias term
    
    @staticmethod
    def _sigmoid(theta):
        # increase stability, redefine sigmoid operation
        return expit(theta) #1/(1+np.exp(-theta))
    
    # vectorized gradient calculation with regularization using L2 Norm
    def _get_gradient(self,X,y):
        ydiff = y-self.predict_proba(X,add_bias=False).ravel() # get y difference
        gradient = np.mean(X * ydiff[:,np.newaxis], axis=0) # make ydiff a column vector and multiply through
        
        gradient = gradient.reshape(self.w_.shape)
        
        # L1 Norm takes absolute value of all w, so multiply w/|w| to C
        if(self.norm == '1'):
            gradient[1:] -= (self.w_[1:]/(abs(self.w_[1:])+.001)) * self.C
        # L2 Norm takes squared value of all w, so multiply 2w to C
        elif(self.norm == '2'):
            gradient[1:] += -2 * self.w_[1:] * self.C
        # L1 and L2 Norm
        elif(self.norm == '3'):
            gradient[1:] -= (self.w_[1:]/(abs(self.w_[1:])+.001)) * self.C
            gradient[1:] += -2 * self.w_[1:] * self.C 
        #else no regularization
        else:
            pass
        
        return gradient
    
    # public:
    def predict_proba(self,X,add_bias=True):
        # add bias term if requested
        Xb = self._add_bias(X) if add_bias else X
        return self._sigmoid(Xb @ self.w_) # return the probability y=1
    
    def predict(self,X):
        return (self.predict_proba(X)>0.5) #return the actual prediction
    
    
    def fit(self, X, y):
        Xb = self._add_bias(X) # add bias term
        num_samples, num_features = Xb.shape
        
        self.w_ = np.zeros((num_features,1)) # init weight vector to zeros
        
        # for as many as the max iterations
        for _ in range(self.iters):
            gradient = self._get_gradient(Xb,y)
            self.w_ += gradient*self.eta # multiply by learning rate 
            # add bacause maximizing 

### Steepest Gradient Descent Solver

In [314]:
# steepest descent from in class examples
# and we can update this to use a line search along the gradient like this:
from scipy.optimize import minimize_scalar
import copy
from numpy import ma # (masked array) this has most numpy functions that work with NaN data.
class LineSearchLogisticRegression(BinaryLogisticRegression):
    
    # define custom line search for problem
    def __init__(self, line_iters=5, **kwds):        
        self.line_iters = line_iters
        # but keep other keywords
        super().__init__(**kwds) # call parent initializer
    
    @staticmethod
    def objective_function(eta,X,y,w,grad,C,self):
        wnew = w - grad*eta
        g = expit(X @ wnew)
        
        if(self.norm == '1'):
            return -np.sum(ma.log(g[y==1]))-np.sum(ma.log(1-g[y==0])) + C*sum(abs(wnew))
        elif(self.norm == '2'):
            return -np.sum(ma.log(g[y==1]))-np.sum(ma.log(1-g[y==0])) + C*sum(wnew**2)
        elif(self.norm == '3'):
            return -np.sum(ma.log(g[y==1]))-np.sum(ma.log(1-g[y==0])) + C*sum(abs(wnew)) + C*sum(wnew**2)
        #else no regularization
        else:
            return -np.sum(ma.log(g[y==1]))-np.sum(ma.log(1-g[y==0]))
        
    def fit(self, X, y):
        Xb = self._add_bias(X) # add bias term
        num_samples, num_features = Xb.shape
        
        self.w_ = np.zeros((num_features,1)) # init weight vector to zeros
        
        # for as many as the max iterations
        for _ in range(self.iters):
            gradient = -self._get_gradient(Xb,y)
            # minimization inopposite direction
            
            # do line search in gradient direction, using scipy function
            opts = {'maxiter':self.line_iters} # unclear exactly what this should be
            res = minimize_scalar(self.objective_function, # objective function to optimize
                                  bounds=(0,self.eta*10), #bounds to optimize
                                  args=(Xb,y,self.w_,gradient,self.C,self), # additional argument for objective function
                                  method='bounded', # bounded optimization for speed
                                  options=opts) # set max iterations
            
            eta = res.x # get optimal learning rate
            self.w_ -= gradient*eta # set new function values
            # subtract to minimize

### Stochastic Gradient Descent Solver

In [315]:
# SGD from class examples
class StochasticLogisticRegression(BinaryLogisticRegression):
    # stochastic gradient calculation 
    def _get_gradient(self,X,y):
        idx = int(np.random.rand()*len(y)) # grab random instance
        ydiff = y[idx]-self.predict_proba(X[idx],add_bias=False) # get y difference (now scalar)
        gradient = X[idx] * ydiff[:,np.newaxis] # make ydiff a column vector and multiply through
        
        gradient = gradient.reshape(self.w_.shape)
        
        # L1 Norm takes absolute value of all w, so multiply w/|w| to C
        if(self.norm == '1'):
            gradient[1:] -= (self.w_[1:]/(abs(self.w_[1:])+.001)) * self.C
        # L2 Norm takes squared value of all w, so multiply 2w to C
        elif(self.norm == '2'):
            gradient[1:] += -2 * self.w_[1:] * self.C
        # L1 and L2 Norm
        elif(self.norm == '3'):
            gradient[1:] -= (self.w_[1:]/(abs(self.w_[1:])+.001)) * self.C
            gradient[1:] += -2 * self.w_[1:] * self.C 
        #else no regularization
        else:
            pass
        
        return gradient

### BFGS Quasi-Newton Method Solver

In [316]:
# BFGS from in class example
from scipy.optimize import fmin_bfgs
from numpy import ma
class BFGSBinaryLogisticRegression(BinaryLogisticRegression):
    
    @staticmethod
    def get_norm(w,C):
        if(self.norm == '1'):
            return C*sum(abs(w))
        elif(self.norm == '2'):
            return C*sum(w**2)
        elif(self.norm == '3'):
            return C*sum(abs(w))+C*sum(w**2)
        #else no regularization
        else:
            return 0
    
    @staticmethod
    def objective_function(w,X,y,C,self):
        g = expit(X @ w)
        
        if(self.norm == '1'):
            return -np.sum(ma.log(g[y==1]))-np.sum(ma.log(1-g[y==0])) + C*sum(abs(w))
        elif(self.norm == '2'):
            return -np.sum(ma.log(g[y==1]))-np.sum(ma.log(1-g[y==0])) + C*sum(w**2)
        elif(self.norm == '3'):
            return -np.sum(ma.log(g[y==1]))-np.sum(ma.log(1-g[y==0])) + C*sum(abs(w)) + C*sum(w**2)
        #else no regularization
        else:
            return -np.sum(ma.log(g[y==1]))-np.sum(ma.log(1-g[y==0]))
        
        # invert this because scipy minimizes, but we derived all formulas for maximzing
        #-np.sum(y*np.log(g)+(1-y)*np.log(1-g))

    @staticmethod
    def objective_gradient(w,X,y,C,self):
        g = expit(X @ w)
        ydiff = y-g # get y difference
        gradient = np.mean(X * ydiff[:,np.newaxis], axis=0)
        gradient = gradient.reshape(w.shape)
        
        # L1 Norm takes absolute value of all w, so multiply w/|w| to C
        if(self.norm == '1'):
            gradient[1:] -= (w[1:]/(abs(w[1:])+.001)) * self.C
        # L2 Norm takes squared value of all w, so multiply 2w to C
        elif(self.norm == '2'):
            gradient[1:] += -2 * w[1:] * self.C
        # L1 and L2 Norm
        elif(self.norm == '3'):
            gradient[1:] -= (w[1:]/(abs(w[1:])+.001)) * self.C
            gradient[1:] += -2 * w[1:] * self.C 
        #else no regularization
        else:
            pass
        
        return -gradient
    
    # just overwrite fit function
    def fit(self, X, y):
        Xb = self._add_bias(X) # add bias term
        num_samples, num_features = Xb.shape
        
        self.w_ = fmin_bfgs(self.objective_function, # what to optimize
                            np.zeros((num_features,1)), # starting point
                            fprime=self.objective_gradient, # gradient function
                            args=(Xb,y,self.C,self), # extra args for gradient and objective function
                            gtol=1e-03, # stopping criteria for gradient, |v_k|
                            maxiter=self.iters, # stopping criteria iterations
                            disp=False)
        
        self.w_ = self.w_.reshape((num_features,1))

### Logistic Regression Implementation

In [317]:
# multi class logistic regression template from class example
class MultiClassLogisticRegression:
    def __init__(self, eta, norm, iterations=20, 
                 C=0.0001, 
                 solver=LineSearchLogisticRegression):
        self.eta = eta
        self.iters = iterations
        self.norm = norm
        self.C = C
        self.solver = solver
        self.classifiers_ = []
        # internally we will store the weights as self.w_ to keep with sklearn conventions
    
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'MultiClass Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained MultiClass Logistic Regression Object'
        
    def fit(self,X,y):
        num_samples, num_features = X.shape
        self.unique_ = np.sort(np.unique(y)) # get each unique class value
        num_unique_classes = len(self.unique_)
        self.classifiers_ = []
        for i,yval in enumerate(self.unique_): # for each unique value
            y_binary = np.array(y==yval).astype(int) # create a binary problem
            # train the binary classifier for this class
            
            lr = self.solver(eta=self.eta,norm=self.norm,iterations=self.iters,C=self.C)
            lr.fit(X,y_binary)

            # add the trained classifier to the list
            self.classifiers_.append(lr)
            
        # save all the weights into one matrix, separate column for each class
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T
        
    def predict_proba(self,X):
        probs = []
        for lr in self.classifiers_:
            probs.append(lr.predict_proba(X).reshape((len(X),1))) # get probability for each classifier
        
        return np.hstack(probs) # make into single matrix
    
    def predict(self,X):
        return np.argmax(self.predict_proba(X),axis=1) # take argmax along rowa

### Training And Testing Based On User Input

In [322]:
%%time
# run the logisitc regression, optimization, regularization and cost based on user input
from ipywidgets import widgets as wd
from sklearn import metrics as mt
    
solver = input("Enter which optimization technique should be used: 1 - Steepest Descent, 2 - Stochastic, 3 - Quasi-Newton(BFGS) ")
norm = input("Enter which regularization technique should be used: 1 - L1 Regularization, 2 - L2 Regularization, 3 - L1 and L2 Regularization, 4 - No Regularization ")

def lr_explor(cost):
    print('Running')
    if(solver == '1'):
        print("Steepest Descent Optimization selected")
        #default solver is Steepest Descent
        lr = MultiClassLogisticRegression(eta=1,
                                  iterations=5, norm=norm,
                                  C=float(cost))
    elif(solver == '2'):
        print("SGD Optimization selected")
        lr = MultiClassLogisticRegression(eta=.01,
                                  iterations=2000, norm=norm,
                                  C=float(cost),
                                  solver=StochasticLogisticRegression
                                 )
    else:
        print("BFGS Quasi-Newton Optimization selected")
        lr = MultiClassLogisticRegression(_,
                                  iterations=20, norm=norm,
                                  C=float(cost),
                                  solver=BFGSBinaryLogisticRegression
                                 )
    acc = []
    for iter_num, (train_indices, test_indices) in enumerate(cv_object.split(X,y)):
        lr.fit(X[train_indices],y[train_indices])  # train object
        y_hat = lr.predict(X[test_indices]) # get test set predictions
        acc.append(mt.accuracy_score(y[test_indices],y_hat))
        
    acc = np.array(acc)
    print(acc.mean(),'+-',2.7*acc.std())
        
wd.interact(lr_explor,cost=list(np.logspace(-4,1,15)),__manual=True)

Enter which optimization technique should be used: 1 - Steepest Descent, 2 - Stochastic, 3 - Quasi-Newton(BFGS) 1
Enter which regularization technique should be used: 1 - L1 Regularization, 2 - L2 Regularization, 3 - L1 and L2 Regularization, 4 - No Regularization 4


interactive(children=(Dropdown(description='cost', options=(0.0001, 0.00022758459260747887, 0.0005179474679231…

CPU times: user 425 ms, sys: 20.1 ms, total: 445 ms
Wall time: 2.8 s


<function __main__.lr_explor(cost)>

### SciKit-Learn Implementation

In [325]:
%%time
# how do we compare now to sklearn?
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr_sk = LogisticRegression(solver='liblinear',n_jobs=1, 
                           multi_class='ovr', C = 1/0.001, 
                           penalty='l2',max_iter=100) 

lr_sk.fit(X,y) # no need to add bias term, sklearn does it internally!!
yhat = lr_sk.predict(X)
print('Accuracy of: ',accuracy_score(y,yhat))

Accuracy of:  0.8580474934036939
CPU times: user 26 ms, sys: 1.98 ms, total: 28 ms
Wall time: 23.4 ms
